In [3]:
import os
import math
import joblib
import pandas
import scipy.io
import numpy as np
import pandas as pd
import ipywidgets as widgets
from sklearn.utils import shuffle
from IPython.display import display
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import roc_auc_score
%env PYDEVD_DISABLE_FILE_VALIDATION=1
%env IPYTHON_XMODE=plain

env: PYDEVD_DISABLE_FILE_VALIDATION=1
env: IPYTHON_XMODE=plain


In [4]:
cwd = os.getcwd()
file = os.path.join(cwd,"Data\\Multiclass_Multioutput_dataset.csv")
data = pd.read_csv(file)
data

,index,Duration,Location1,Location2,Score,Doctor,PatientTypes,Knowledge,Type2diabetes,Arthritis,Depression,Cost,Weight,Interaction,Medicine1,Medicine2,Medicine3
0,1,177,40.7009,-73.9461,0.9,Family medicine,Heart disease & Type-2 Diabetes,Knowledgeable,1,0,0,1,1,0,Metformin,None,None
1,2,190,40.7434,-73.0761,0.7,Family medicine,Heart disease,Knowledgeable,1,0,0,1,1,0,Victoza,None,None
2,3,271,40.7009,-73.9461,1.0,Family medicine,Depression,Very knowledgeable,1,0,0,1,1,0,Metformin,None,None
3,4,268,40.7009,-73.9461,1.0,Family medicine,Cancer,Not very knowledgeable,1,0,0,1,1,0,Victoza,None,None
4,5,303,40.7009,-73.9461,0.9,Family medicine,Heart disease & Type-2 Diabetes,Knowledgeable,1,0,0,1,1,0,Victoza,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926,3927,722,32.8209,-97.1765,1.0,Family medicine,Heart disease & Type-2 Diabetes,Very knowledgeable,1,1,1,1,1,1,Victoza,Hydroxychloroquine,Escitalopram
3927,3928,795,32.8209,-97.1765,1.0,Family medicine,Heart disease & Type-2 Diabetes,Very knowledgeable,1,1,1,1,1,1,Metformin,Hydroxychloroquine,Escitalopram
3928,3929,1040,46.1478,-122.9711,1.0,Family medicine,Type-2 Diabetes & Depression,Knowledgeable,1,1,1,1,1,1,Empagliflozin,Ibuprofen,Selegiline
3929,3930,928,32.8209,-97.1765,1.0,General internal medicine,Type-2 Diabetes & Depression,Knowledgeable,1,1,1,1,1,1,Empagliflozin,Hydroxychloroquine,Escitalopram


In [2]:
cwd = os.getcwd()
file = os.path.join(cwd,"Data\\Multiclass_Multioutput_dataset.csv")
data = pd.read_csv(file)
data.Medicine1.replace(to_replace=['None'], value=np.nan, inplace=True)
data.Medicine2.replace(to_replace=['None'], value=np.nan, inplace=True)
data.Medicine3.replace(to_replace=['None'], value=np.nan, inplace=True)

df = data.fillna('')
ord_enc = OrdinalEncoder()

#data["Medicine"] = data['Medicine1'].astype(str)+"-"+data['Medicine2'].astype(str)+"-"+data['Medicine3'].astype(str)

X=df.drop(['index','Medicine1','Medicine2','Medicine3'], axis=1)
X=ord_enc.fit_transform(X)
y = df[['Medicine1','Medicine2','Medicine3']]
y=ord_enc.fit_transform(y)

training, testing, training_labels, testing_labels = train_test_split(X, y, test_size = .30, random_state = 108)

forest = RandomForestClassifier(n_estimators=100, max_depth=12,random_state=2)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=2)
multi_target_forest.fit(training, training_labels)

accuracy = multi_target_forest.score(training, training_labels)*100
print("Accuracy:", accuracy,"%")

# Save the trained model to a file
joblib.dump(multi_target_forest, 'Data\\Models\\trained_model.joblib')

# customTesting = [[ 6., 31., 38.,  0.,  3., 21.,  0.,  1.,  1.,  0.,  0.,  1.,  1.]]
# prediction = multi_target_forest.predict(customTesting)

#medicine=["Medicine1:","Medicine2:","Medicine3:"]
# meds = []
# for i in range(len(prediction)):
#     #print(prediction[i])
#     for j in range(len(prediction[i])):  
#         #print(medicine[j],ord_enc.categories_[j][round(prediction[i][j])])
#         meds.append(ord_enc.categories_[j][round(prediction[i][j])])
    
# for item in meds:
#     if item != '':
#         print(item)

Accuracy: 93.0934205743366 %


['Data\\Models\\trained_model.joblib']

NameError: name 'data' is not defined

In [3]:
testing_length = len(testing)
print("Length of testing dataset:", testing_length)

Length of testing dataset: 1180


In [4]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import roc_auc_score

knn = KNeighborsRegressor(algorithm='kd_tree', leaf_size=20, metric='minkowski',
                          metric_params=None, n_jobs=15, n_neighbors=2, p=3,
                          weights='uniform')

regr = MultiOutputRegressor(knn)
regr.fit(training, training_labels)

preds = regr.predict(testing)

accuracy = regr.score(training, training_labels)*100
print("Accuracy:", accuracy,"%")

# Save the trained model to a file
joblib.dump(regr, 'Data\\Models\\trained_knn.joblib')

Accuracy: 75.94077175432264 %


['Data\\Models\\trained_knn.joblib']

In [5]:
# print(X,y)

In [6]:
# Later, when you want to make predictions using the saved model
# Load the saved model from the file
loaded_model = joblib.load('trained_model.joblib')

customTesting = [[ 6., 31., 38.,  0.,  3., 21.,  0.,  1.,  1.,  0.,  0.,  1.,  1.]]
prediction = loaded_model.predict(customTesting)
print(prediction)

[[1. 3. 0.]]


In [7]:
from ipywidgets import widgets, Layout
from IPython.display import display, HTML
from ipywidgets import widgets, interact
from IPython.display import display, HTML

loaded_model = joblib.load('trained_model.joblib')
loaded_knn = joblib.load('trained_knn.joblib')

# Create checkbox widgets
disease1 = widgets.Checkbox(value=False, description='Type 2 Diabetes')
disease2 = widgets.Checkbox(value=False, description='Arthritis')
disease3 = widgets.Checkbox(value=False, description='Depression')
factor1 = widgets.Checkbox(value=False, description='Weight')
factor2 = widgets.Checkbox(value=False, description='Cost')
factor3 = widgets.Checkbox(value=False, description='Interaction')

style = """
        <style>
        .custom-toggle-buttons .widget-toggle-button {
            width: 100%;
            max-width: 100%;
        }
        .custom-toggle-buttons .widget-toggle-button:not(.mod-active) {
            background-color: transparent;
            border-color: transparent;
            color: inherit;
            border-color: #ccc;
            border-width: 1px;
            border-style: solid;
        }
        .custom-toggle-buttons .widget-toggle-button.mod-active {
            background-color: #28a745;  /* Success button color */
            border-color: #28a745;
            color: #fff;
        }
        </style>
        """
toggle_button = widgets.ToggleButtons(
    options=['Random forest', 'K-Nearest Neighbor'],
    button_style='info',  # Apply a button style to change the color
    layout=Layout(width='auto'),  # Set the width to auto
    style={'button_width': '30%'}  # Set the button width to initial
)

toggle_button.add_class('custom-toggle-buttons')
display(HTML(style))
display(toggle_button)

# Creating a button widget
button = widgets.Button(description='Submit')

# Create an output widget
output = widgets.Output()

# Display the checkboxes
print("Select all the diseases that apply: ")
display(disease1, disease2, disease3)

# Display the checkboxes
print("Select all the factors that influence your medication plan: ")
display(factor1, factor2, factor3)
display(button)

def handle_submit_button(button_click):
    with output:
        output.clear_output()
        meds = []
        flag = 0;
        
        selected_value = toggle_button.value
        if selected_value == 'Random forest':
            flag = 0;
        else:
            flag = 1;
        
#         selected_diseases = [disease.description for disease in [disease1, disease2, disease3] if disease.value]
#         selected_factors = [factor.description for factor in [factor1, factor2, factor3] if factor.value]
        diabetes = 1. if disease1.value else 0.
        arthritis = 1. if disease2.value else 0.
        depression = 1. if disease3.value else 0.
        weight = 1. if factor1.value else 0.
        cost = 1. if factor2.value else 0.
        interaction = 1. if factor3.value else 0.

        customTesting = [[ 6., 31., 38.,  0.,  3., 21.,  0.,  diabetes,  arthritis,  depression,  weight,  cost,  interaction]]
        if flag == 0:
            prediction = loaded_model.predict(customTesting)
        else:
            prediction = loaded_knn.predict(customTesting)
        
        meds_buttons = []
        print("The best medication suggestion is: ")
        for i in range(len(prediction)):
            for j in range(len(prediction[i])):  
                meds.append(ord_enc.categories_[j][round(prediction[i][j])])
    
        for item in meds:
             if item != '':
                meds_buttons.append(widgets.Button(description=item, button_style='success'))
        display(*meds_buttons)

button.on_click(handle_submit_button)
display(output)


ToggleButtons(_dom_classes=('custom-toggle-buttons',), button_style='info', layout=Layout(width='auto'), optio…

Select all the diseases that apply: 


Checkbox(value=False, description='Type 2 Diabetes')

Checkbox(value=False, description='Arthritis')

Checkbox(value=False, description='Depression')

...

Checkbox(value=False, description='Weight')

Checkbox(value=False, description='Cost')

Checkbox(value=False, description='Interaction')

Button(description='Submit', style=ButtonStyle())

Output()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df.drop(['index','Medicine1','Medicine2','Medicine3'], axis=1)
X = ord_enc.fit_transform(X)
y = df[['Medicine1','Medicine2','Medicine3']]
y = ord_enc.fit_transform(y)

# Standardize the numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform PCA to reduce the dimensionality
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Create a TensorFlow model for the visualization
model = Sequential()
model.add(Dense(3, input_shape=(3,), activation='linear'))  # 3 components from PCA
model.compile(optimizer='adam', loss='mse')

# Create a TensorBoard callback
logdir = "./logs"
tensorboard_callback = TensorBoard(log_dir=logdir)

# Create a ModelCheckpoint callback to save checkpoints
checkpoint_callback = ModelCheckpoint('./checkpoints/model-{epoch:02d}.ckpt', save_weights_only=True)

# Fit the model to the PCA-transformed data
model.fit(X_pca, X_pca, epochs=100, callbacks=[tensorboard_callback, checkpoint_callback])

In [ ]:
# Launch TensorBoard
#%load_ext tensorboard
%tensorboard --logdir=./logs
%reload_ext tensorboard

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import joblib

loaded_model = joblib.load('trained_model.joblib')
loaded_knn = joblib.load('trained_knn.joblib')

# Select a sample of indices for plotting
sample_indices = [0, 1, 2, 3, 4]

# Get the predictions from Model 1 and Model 2 for the selected indices
model1_predictions = loaded_model.predict(testing)[sample_indices]
model2_predictions = loaded_knn.predict(testing)[sample_indices]

# Generate x-axis values for plotting
x_values = np.arange(len(sample_indices))

# Plot the predictions from Model 1
model1_lines = plt.plot(x_values, model1_predictions, label="Model 1")

# Plot the predictions from Model 2
model2_lines = plt.plot(x_values, model2_predictions, label="Model 2")

# Add labels and title
plt.xlabel('Sample Index')
plt.ylabel('Predicted Value')
plt.title('Comparison of Model Predictions')

# Add a legend with only the lines from Model 1 and Model 2
lines = model1_lines + model2_lines
labels = [line.get_label() for line in lines]
plt.legend(lines, labels)

# Display the graph
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import joblib
from mpl_toolkits.mplot3d import Axes3D

loaded_model = joblib.load('trained_model.joblib')
loaded_knn = joblib.load('trained_knn.joblib')

# Get the predictions from Model 1 and Model 2 for the entire testing dataset
model1_predictions = loaded_model.predict(testing)
model2_predictions = loaded_knn.predict(testing)

# Extract the desired number of predictions for plotting
num_predictions = 1180
model1_predictions = model1_predictions[:num_predictions]
model2_predictions = model2_predictions[:num_predictions]

# Create figure and 3D axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Extract the individual multioutput values for each model
model1_pred_val1 = model1_predictions[:, 0]
model1_pred_val2 = model1_predictions[:, 1]
model1_pred_val3 = model1_predictions[:, 2]

model2_pred_val1 = model2_predictions[:, 0]
model2_pred_val2 = model2_predictions[:, 1]
model2_pred_val3 = model2_predictions[:, 2]

# Plot the predictions from Model 1
ax.scatter(model1_pred_val1, model1_pred_val2, model1_pred_val3, label="Random Forest")

# Plot the predictions from Model 2
ax.scatter(model2_pred_val1, model2_pred_val2, model2_pred_val3, label="K-nearest Neighbors")

# Set labels for each axis
ax.set_xlabel('Medicine for diabetes')
ax.set_ylabel('Medicine for arthritis')
ax.set_zlabel('Medicine for depression')

# Add a legend
ax.legend()

# Display the 3D plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have loaded the predictions from the two models as `model1_predictions` and `model2_predictions`

# Generate x-axis values
x_values = np.arange(len(model1_predictions))

# Plot the predictions from Model 1
plt.plot(x_values, model1_predictions, label='Model 1')

# Plot the predictions from Model 2
plt.plot(x_values, model2_predictions, label='Model 2')

# Add labels and title
plt.xlabel('Sample Index')
plt.ylabel('Prediction')
plt.title('Comparison of Model Predictions')

# Add a legend
plt.legend()

# Show the plot
plt.show()
